по терминологии: чтобы не возникло путаницы в понятии вершины, я буду называть вершину графа вертексом, а вершину в синтаксическом дереве вершиной
а ребро графа либо ребром, либо эджом. чанк - это именная группа

In [ ]:
import spacy
import spacy.cli

spacy.cli.download("en_core_web_lg")

In [8]:
nlp_en = spacy.load("en_core_web_lg")
doc = nlp_en("Colorless green ideas sleep furiously with blue elephants")

In [32]:
from spacy import displacy

In [33]:
displacy.render(doc, style='dep')

In [ ]:
def make_vertex():
    pass

- бежим по чанкам (чанк - вертекс)
- у каждого чанка ищем вершину (она будет эджом)
- проверяем, есть ли уже в графе этот эджс с такой же дочерней вершиной. если есть, то значит это тот же эдж и мы просто добавляем ему второй вертекс. если нет, то создаём новый эдж

In [ ]:
graph = [] # это будет объект класса граф но пока пусть будет список
for chunk in doc.noun_chunks:
    edge = chunk.root.head
    # если в графе есть такой вертекс:
        # проверяем, есть ли уже у этого вертекса такой чанк в качестве вершины. если нет, то 
            # это значит, что в графе есть такой эдж с заполненной одной стороной. мы заполняем вторую. добавляем к 

sleep [ideas, furiously, with]
with [elephants]


пока не знаю как реализовать односторонние и двухсторонние эджи